In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import axes3d
from matplotlib import animation
from matplotlib import cm
from IPython.display import HTML
from ipywidgets import AppLayout, IntSlider, FloatSlider, interact
from scipy.special import sph_harm
import pdb

Legendre polynomials:

$$
P_\ell^m(x) = \frac{(-\ell)^m}{2^\ell \ell!}(1 - x^2)^{m/2} \frac{d^{\ell+m}}{dx^{\ell+m}}(x^2 - \ell)^\ell
$$

Note that this formula is only well-defined and nonzero for $\ell \geq 0$ and $m$ integers such that $|m| \leq \ell$. We can ensure these conditions by requiring that solutions are periodic in $\theta$ and $\phi$

The general solutions for the Legendre polynomials are the linearly dependent **spherical harmonics**:

$$
Y_\ell^m(\theta, \phi) = \sqrt{\frac{2 \ell + 1 (\ell - m) !}{ 4 \pi (\ell + m) !)}} P^m_\ell (cos \theta) e ^{i m \phi}
$$

In [4]:
def assoc_legendre_polynomial(l, m, x):
    term1 = (-l ** m) / (2 ** l) * np.math.factorial(l)
    term2 = (1 - x ** 2) ** (m / 2)
    term3 = (d ** (l + m)) / (d * x**(l + m))
    term4 = (x ** 2 - l) ** l
    return term1 * term2 * term3 * term4

In [28]:
def assoc_legendre_polynomial(l, m, theta):
    numer = 2 * l + 1 * np.math.factorial(l - m)
    denom = 2 * np.math.factorial(l + m)
    return np.sqrt(numer / denom)

In [5]:
def spherical_harmonics(l, m, theta, phi):
    fact1 = np.math.factorial(l - m)
    fact2 = np.math.factorial(l + m)
    term1 = np.sqrt((2* l + fact1) / (4 * np.pi * fact2))
    term2 = assoc_legendre_polynomial(l, m, np.cos(theta))
    term3 = np.exp(1j * m * phi)
    return term1 * term2 * term3

In [21]:
def spherical_to_cartesian(r, phi, theta):
    """
    r: radius in {0, inf}
    phi: polar coordinate in {0, 2pi}
    theta: azimuthal coordinate in {0, pi}
    """
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)
    return x, y, z

In [27]:
# phi running from 0 to pi and tta from 0 to pi
phi = np.linspace(0, 2 * np.pi, 25)
theta = np.linspace(0, np.pi, 25)

# meshgrid to generate points
phi, theta = np.meshgrid(phi, theta)

# THIS IS THE FUNCTION
Y = np.cos(theta)
# finally all things in cartesian co-ordinate system
# Note that "Y" is acting as "r"
x, y, z = spherical_to_cartesian(np.abs(Y), phi, theta)

# plotting :-
fig = plt.figure()
ax = fig.add_subplot( 111 , projection='3d')
ax.plot_surface(x, y, z, linewidth = 0.5, edgecolors = 'k', facecolors=cm.jet(Y))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [2]:
phi = np.linspace(0, 2 * np.pi, 25)
theta = np.linspace(0, np.pi, 25)

# meshgrid to generate points
phi, theta = np.meshgrid(phi, theta)

In [36]:
harm = sph_harm(1, 1, phi, theta)

In [48]:
x, y, z = spherical_to_cartesian(np.abs(harm), phi, theta)

In [49]:
# plotting :-
fig = plt.figure()
ax = fig.add_subplot( 111 , projection='3d')
ax.plot_surface(x, y, z, linewidth = 0.5, edgecolors = 'k', facecolors=cm.jet(Y))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
# plt.rc('text', usetex=True)

# Grids of polar and azimuthal angles
theta = np.linspace(0, np.pi, 100)
phi = np.linspace(0, 2*np.pi, 100)
# Create a 2-D meshgrid of (theta, phi) angles.
theta, phi = np.meshgrid(theta, phi)
# Calculate the Cartesian coordinates of each point in the mesh.
xyz = np.array([np.sin(theta) * np.sin(phi),
                np.sin(theta) * np.cos(phi),
                np.cos(theta)])

def plot_Y(ax, el, m):
    """Plot the spherical harmonic of degree el and order m on Axes ax."""

    # NB In SciPy's sph_harm function the azimuthal coordinate, theta,
    # comes before the polar coordinate, phi.
    Y = sph_harm(abs(m), el, phi, theta)

    # Linear combination of Y_l,m and Y_l,-m to create the real form.
    if m < 0:
        Y = np.sqrt(2) * (-1)**m * Y.imag
    elif m > 0:
        Y = np.sqrt(2) * (-1)**m * Y.real
    Yx, Yy, Yz = np.abs(Y) * xyz

    # Colour the plotted surface according to the sign of Y.
    cmap = plt.cm.ScalarMappable(cmap=plt.get_cmap('PRGn'))
    cmap.set_clim(-0.5, 0.5)

    ax.plot_surface(Yx, Yy, Yz,
                    facecolors=cmap.to_rgba(Y.real),
                    rstride=2, cstride=2)

    # Draw a set of x, y, z axes for reference.
    ax_lim = 0.5
    ax.plot([-ax_lim, ax_lim], [0,0], [0,0], c='0.5', lw=1, zorder=10)
    ax.plot([0,0], [-ax_lim, ax_lim], [0,0], c='0.5', lw=1, zorder=10)
    ax.plot([0,0], [0,0], [-ax_lim, ax_lim], c='0.5', lw=1, zorder=10)
    # Set the Axes limits and title, turn off the Axes frame.
    ax.set_title(r'$Y_{{{},{}}}$'.format(el, m))
    ax_lim = 0.5
    ax.set_xlim(-ax_lim, ax_lim)
    ax.set_ylim(-ax_lim, ax_lim)
    ax.set_zlim(-ax_lim, ax_lim)
    ax.axis('off')

fig = plt.figure(figsize=plt.figaspect(1.))
ax = fig.add_subplot(projection='3d')
l, m = 3, 0
plot_Y(ax, l, m)
plt.savefig('Y{}_{}.png'.format(l, m))
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
el_max = 3
figsize_px, DPI = 800, 100
figsize_in = figsize_px / DPI
fig = plt.figure(figsize=(figsize_in, figsize_in), dpi=DPI)
spec = gridspec.GridSpec(ncols=2*el_max+1, nrows=el_max+1, figure=fig)
for el in range(el_max+1):
    for m_el in range(-el, el+1):
        ax = fig.add_subplot(spec[el, m_el+el_max], projection='3d')
        plot_Y(ax, el, m_el)
plt.tight_layout()
# plt.savefig('sph_harm.png')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …